<a href="https://colab.research.google.com/github/kenzou-eu/EP1/blob/main/EP_Num%C3%A9rico_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def AlgoritmoQR(n,A,B,V,e):
    k=0
    for i in range(n-1):
        while abs(B[i])>e:
            cos=[]
            sen=[]
            C=[]
            for j in range(n-1):
                C.append(B[j])
            d=(A[n-2]-A[n-1])/2
            if d>=0:
                h=A[n-1]+d-pow(d*d+B[n-2]*B[n-2],1/2)
            if d<0:
                h=A[n-1]+d+pow(d*d+B[n-2]*B[n-2],1/2)
            for j in range(n):
                A[j]=A[j]-h
            for q in range(n-1):
                if q!=(n-2):
                    cos.append(A[q]/pow(A[q]*A[q]+B[q]*B[q],1/2)) #ck
                    sen.append(-B[q]/pow(A[q]*A[q]+B[q]*B[q],1/2))
                    A.insert(q+1,(cos[q]*A[q]-sen[q]*B[q]))
                    A.pop(q)
                    A.insert(q+2,(sen[q]*C[q]+cos[q]*A[q+1]))
                    C.insert(q+1,(cos[q]*C[q]-sen[q]*A[q+1]))
                    A.pop(q+1)
                    C.pop(q)
                    C.insert(q+2,(cos[q]*C[q+1]))
                    C.pop(q+1)
                if q==(n-2):
                    cos.append(A[q]/pow(A[q]*A[q]+B[q]*B[q],1/2))
                    sen.append(-B[q]/pow(A[q]*A[q]+B[q]*B[q],1/2))
                    A.insert(q+1,(cos[q]*A[q]-sen[q]*B[q]))
                    A.pop(q)
                    A.insert(q+2,(sen[q]*C[q]+cos[q]*A[q+1]))
                    C.insert(q+1,(cos[q]*C[q]-sen[q]*A[q+1]))
                    A.pop(q+1)
                    C.pop(q)
            for q in range(n-1):
                A.insert(q+1,(cos[q]*A[q]-sen[q]*C[q]))
                A.pop(q)
                A.insert(q+2,(cos[q]*A[q+1]))
                B.insert(q+1,(-sen[q]*A[q+1]))
                A.pop(q+1)
                B.pop(q)
                for m in range(n):
                    V[m].insert(q+1,(cos[q]*V[m][q]-sen[q]*V[m][q+1]))
                    V[m].insert(q+3,(sen[q]*V[m][q]+cos[q]*V[m][q+2]))
                    V[m].pop(q)
                    V[m].pop(q+1)
            for j in range(n):
                A[j]=A[j]+h
            k=k+1

n=int(input('Qual a ordem da matriz?'))

A=[]
for i in range(n):
    if i==0:
        a=float(input('Digite o primeiro elemento da diagonal principal:'))
        A.append(a)
    else:
        a=float(input('Digite o próximo elemento da diagonal principal:'))
        A.append(a)
        
B=[]
for i in range(n-1):
    if i==0:
        b=float(input('Digite o primeiro elemento da sub-diagonal:'))
        B.append(b)
    else:
        b=float(input('Digite o próximo elemento da sub-principal:'))
        B.append(b)
        
V=[]
def v(n):
    v0=[]
    for i in range(n):
        v0.append(0)
    return(v0)
for i in range(n):
    V.append(v(n))
for i in range(n):
    V[i][i]=1


e=float(input('Qual valor do erro?'))

AlgoritmoQR(n,A,B,V,e)
print('A=',A)
print('B=',B)
print('V=',V)

### Início do programa


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
pi = math.pi

In [20]:
def ck(a, b, k):
    dividend = a[k]
    divisor = math.sqrt(pow(a[k], 2) + pow(b[k], 2))
    c = dividend/divisor
    return c

def sk(a, b, k):
    dividend = -b[k]
    divisor = math.sqrt(pow(a[k], 2) + pow(b[k], 2))
    s = (dividend/divisor)
    return s

def sgn(d):
    if d >=0:
        return 1
    else:
        return -1

def dk(alfa, n): #alfa(i,k) é o coeficiente i da diagonal principal da matriz A na interação k, beta é da outra diagonal
    valuedk = (alfa[n-1] - alfa[n])/2
    return valuedk

def uk(alfa, beta, d, n, k, desloc): #Heurística de wilkinson
    if k!= 0 and desloc == 1:
      valueuk = alfa[n] + dk(alfa, n) - sgn(d)*(math.sqrt(pow(dk(alfa,n), 2) + pow(beta[n-1],2)))
      return valueuk
    if k==0 or desloc == 0:
      return 0

def matA(n): #A é a matriz da diagonal principal
    A = []
    for i in range(n):
        if i==0:
            a=float(input('Digite o primeiro elemento da diagonal principal:'))
            A.append(a)
        else:
            a=float(input('Digite o próximo elemento da diagonal principal:'))
            A.append(a)
    return A

def matB(n): #B é a matriz da sub-diagonal 
    B = []
    for i in range(n-1):
        if i==0:
            b=float(input('Digite o primeiro elemento da sub-diagonal: '))
            B.append(b)
        else:
            b=float(input('Digite o próximo elemento da sub-diagonal: '))
            B.append(b)
    return B

def v(n):
    v0=[]
    for i in range(n):
        v0.append(0)
    return(v0)

def handleV(n):
  V=[]
  for i in range(n):
      V.append(v(n))
  for i in range(n):
      V[i][i]=1
  return V



In [15]:
def metodoQR(n, A, B, V, erro, desloc):
    k = 0
    for i in range(n-1):
        while abs(B[i]) > erro:
            cos = []
            sen = []
            C=[]
            for j in range(n-1):
                C.append(B[j])
            valueDK = dk(A, n-1)
            valueUK = uk(A, B, valueDK, n-1, k, desloc)
            for j in range(n):
                A[j] = A[j] - valueUK
            for q in range(n-1):
                if q != (n-2):
                    cos.append(ck(A,B,q))
                    sen.append(sk(A,B,q))
                    A.insert(q+1,(cos[q]*A[q]-sen[q]*B[q]))
                    A.pop(q)
                    A.insert(q+2,(sen[q]*C[q]+cos[q]*A[q+1]))
                    C.insert(q+1,(cos[q]*C[q]-sen[q]*A[q+1]))
                    A.pop(q+1)
                    C.pop(q)
                    C.insert(q+2,(cos[q]*C[q+1]))
                    C.pop(q+1)
                if q == (n-2):
                    cos.append(ck(A,B,q))
                    sen.append(sk(A,B,q)) 
                    A.insert(q+1,(cos[q]*A[q]-sen[q]*B[q]))
                    A.pop(q)
                    A.insert(q+2,(sen[q]*C[q]+cos[q]*A[q+1]))
                    C.insert(q+1,(cos[q]*C[q]-sen[q]*A[q+1]))
                    A.pop(q+1)
                    C.pop(q)
            for q in range(n-1):
                A.insert(q+1,(cos[q]*A[q]-sen[q]*C[q]))
                A.pop(q)
                A.insert(q+2,(cos[q]*A[q+1]))
                B.insert(q+1,(-sen[q]*A[q+1]))
                A.pop(q+1)
                B.pop(q)
                for m in range(n):
                    V[m].insert(q+1,(cos[q]*V[m][q]-sen[q]*V[m][q+1]))
                    V[m].insert(q+3,(sen[q]*V[m][q]+cos[q]*V[m][q+2]))
                    V[m].pop(q)
                    V[m].pop(q+1)
            for j in range(n):
                A[j] = A[j] + valueUK
            k = k+1
    return A, B, V, k
    # print('A =',A)
    # print('B =',B)
    # print('V =')
    # for i in range(n):
    #   print(V[i])
    # print("O numero de interações foi: ", k)

In [30]:
# %%
desloc = int(input("Deseja realizar o QR com deslocamento? Se sim, digite 1, se não, digite 0. "))
n = int(input("Qual a ordem da Matriz? ")) #tamanho da matriz
# A = matA(n) #cria um vetor com os elementos da diagonal principal
# B = matB(n) #cria um vetor com os elementos da sub-diagonal 
# erro = float(input("Qual o valor do erro? Digite na forma 1.0e-6 "))

A = []
for i in range(n):
  A.append(2)
B = []
for i in range(n-1):
  B.append(-1)
erro = float(1.0e-6)

V = handleV(n)

A, B, V, k = metodoQR(n, A, B, V, erro, desloc)
print('A =',A)
print('V =')
for i in range(n):
    print(V[i])
print("O numero de interações foi: ", k)



Deseja realizar o QR com deslocamento? Se sim, digite 1, se não, digite 0. 1
Qual a ordem da Matriz? 4
A = [3.6180339887490436, 2.6180339887507444, 1.3819660112501082, 0.38196601125010526]
V =
[0.3717485886598342, 0.6015006124930082, 0.6015009550075102, 0.3717480344601846]
[-0.6015012975216081, -0.3717474802601965, 0.3717480344602072, 0.601500955007546]
[0.6015006124929727, -0.37174858865985694, -0.3717480344601624, 0.6015009550075457]
[-0.3717474802602187, 0.6015012975215729, -0.6015009550075818, 0.37174803446018456]
O numero de interações foi:  39


In [9]:
def autovalores(n):
  A = []
  for i in range(n):
    a = 2*(1-math.cos(((i+1)*pi)/(n+1)))
    A.insert(0,a)
  return A

def autovetores(n):
  V = []
  a = 0
  for i in range(n):
    vector = []
    for j in range(n):
      v = math.sin(((j+1)*(i+1)*pi)/(n+1))
      vector.insert(0, v)
      a = a + v**2
    normalized = vector/np.sqrt(a)
    a = 0
    V.append(normalized)
  return V


n = int(input("Qual a ordem da Matriz? ")) #tamanho da matriz
Vteorico = autovetores(n)
Ateorico = autovalores(n)
print("O A teórico é: ", Ateorico)
display("O V teórico é: ", Vteorico)

Qual a ordem da Matriz? 4
O A teórico é:  [3.618033988749895, 2.618033988749895, 1.381966011250105, 0.3819660112501051]


'O V teórico é: '

[array([0.37174803, 0.60150096, 0.60150096, 0.37174803]),
 array([-0.60150096, -0.37174803,  0.37174803,  0.60150096]),
 array([ 0.60150096, -0.37174803, -0.37174803,  0.60150096]),
 array([-0.37174803,  0.60150096, -0.60150096,  0.37174803])]

In [56]:
# Autovetores representam os modos naturais de vibração
# Autovalores determinam as frequências

#ponto de parada é t=60
def k(n):
    ki = (40 + 2*n)
    return ki

def mainDiag(n,m):
    A=[]
    for i in range(1,n+1):
        a = (k(i) + k(i+1))/m
        A.append(a)
    return A

def subDiag(n,m):
    B = []
    for i in range(2,n+1):
        b = -k(i)/m
        B.append(b)
    return B

def a(x0, n):
    mat_a=[]
    for i in range(n):
        value = x0[i]
        mat_a.append(value)
    return mat_a

def handley0(x0, V):
    Vt = np.transpose(V)
    y0 = np.dot(Vt, x0)
    return y0

def functionY(t, a, n, A):
      Y =[]
      for i in range(n):
        y = a[i]*math.cos(math.sqrt(A[i]*t))
        Y.append(y)
      return Y

def functionX(t, a, n, A, V):
    Y = functionY(t, a, n, A)
    X = np.dot(V, Y)
    somaX = sum(X)
    return somaX


x0 = [-2, -3, -1, -3, -1]
n = 5 #Quantidade de massas
m = 2 #Massa das massas
desloc = 1
t = 60
erro = 1.0e-6
V = handleV(n)
A = mainDiag(n,m)
B = subDiag(n,m)

A, B, V, k = metodoQR(n, A, B, V, erro, desloc)
print("A", A)
print("V", V)
y0 = handley0(x0, V)
value_a = a(y0, n)

X = []
for i in range(1, t+1):
    xi = functionX(i, value_a, n, A, V)
    X.append(xi)
print("X", X)
eixo_y = []
for i in range(t+1):
  eixo_y.append(i)

for i in range(n):
  eixo_x = []
  eixo_x.append(x0[i])
  for j in range(t): 
      eixo_x.append(X[j][i])
  print(eixo_x)
  plt.plot(eixo_x, eixo_y)
  plt.show()

A [88.44500562036662, 70.11383113050884, 46.773186340059276, 23.39863315456126, 6.269343754503707]
V [[0.18933468291544792, 0.47497609407951336, 0.5988223982052299, 0.5325039524890409, 0.3104856903136783], [-0.39110523908448613, -0.5853827917637471, -0.10270311331950498, 0.4744456963356248, 0.5183792345480607], [0.5576611177206107, 0.18485783142726275, -0.5648687392563447, -0.06375701825971554, 0.5759340326601525], [-0.5882019781905723, 0.38295966928649927, 0.09308548584216841, -0.5173751580265936, 0.4806436962399047], [0.3927105555511234, -0.5008933729046883, 0.5505653509427288, -0.4686137111648992, 0.2686321510699377]]
X [6.903196352493397, 9.706933388435045, 2.80252621025519, -2.8515050506652693, -6.990359744393031, -9.287927403005076, -9.275622257869554, -7.255300632762449, -3.4258593809390603, 1.1916771580244379, 4.930180624701314, 7.033552299024655, 8.072359337033468, 8.80738440735466, 9.20840922214571, 8.65898530514836, 6.774435343726655, 3.8341633473589005, 0.5397541142905452, 

IndexError: ignored